In [1]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from gpt import SingleHeadAttention, MultiHeadAttention, FeedForward, Block, GPT

In [3]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [4]:
torch.manual_seed(2525)

In [5]:
with open('tiny-shakespeare.txt', 'r', encoding='utf-8') as f:
    text=f.read()

In [6]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [7]:
len(text)

1115394

In [8]:
chars = sorted(list(set(text)))
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
vocab_size = len(chars)
vocab_size

65

In [10]:
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [11]:
print(encode("Sword in Mouth, Fire Eyes"))
print(decode(encode("Hello, World!")))

[31, 61, 53, 56, 42, 1, 47, 52, 1, 25, 53, 59, 58, 46, 6, 1, 18, 47, 56, 43, 1, 17, 63, 43, 57]
Hello, World!


In [12]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [13]:
print(data[:200])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [14]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [15]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

In [16]:
xb, yb = get_batch('train')

In [17]:
print("inputs:")
xb.shape, xb

inputs:


(torch.Size([16, 32]),
 tensor([[56, 42, 57, 46, 47, 54,  1, 54, 50, 43, 39, 57, 43,  1, 58, 53,  1, 39,
          57, 49,  8,  0,  0, 20, 13, 31, 32, 21, 26, 19, 31, 10],
         [ 1, 51, 63,  1, 51, 47, 57, 44, 53, 56, 58, 59, 52, 43,  5, 57,  1, 41,
          56, 53, 57, 57, 11,  0, 13, 63,  6,  1, 39, 63,  6,  1],
         [59, 43,  8,  0, 21,  1, 61, 53, 59, 50, 42,  1, 52, 53, 58,  7,  7, 58,
          46, 53, 59, 45, 46,  1,  5, 58, 47, 57,  1, 51, 63,  1],
         [ 1, 51, 43,  1, 42, 53,  1, 40, 39, 41, 49,  1, 56, 43, 41, 43, 47, 60,
          43,  1, 58, 46, 43,  1, 44, 50, 53, 59, 56,  1, 53, 44],
         [ 1, 58, 56, 59, 43, 12,  0,  0, 17, 24, 14, 27, 35, 10,  0, 27,  1, 58,
          46, 53, 59,  1, 41, 39, 47, 58, 47, 44, 44,  2,  1, 27],
         [ 1, 58, 46, 39, 58,  1, 58, 56, 59, 57, 58,  1, 58, 53,  1, 58, 46, 43,
          43,  8,  0,  0, 15, 13, 25, 21, 24, 24, 27, 10,  0, 20],
         [53, 49,  1, 58, 53,  1, 55, 59, 43, 52, 41, 46,  1, 47, 58,  6,  0, 31,
 

In [18]:
print("targets:")
yb.shape, yb

targets:


(torch.Size([16, 32]),
 tensor([[42, 57, 46, 47, 54,  1, 54, 50, 43, 39, 57, 43,  1, 58, 53,  1, 39, 57,
          49,  8,  0,  0, 20, 13, 31, 32, 21, 26, 19, 31, 10,  0],
         [51, 63,  1, 51, 47, 57, 44, 53, 56, 58, 59, 52, 43,  5, 57,  1, 41, 56,
          53, 57, 57, 11,  0, 13, 63,  6,  1, 39, 63,  6,  1, 44],
         [43,  8,  0, 21,  1, 61, 53, 59, 50, 42,  1, 52, 53, 58,  7,  7, 58, 46,
          53, 59, 45, 46,  1,  5, 58, 47, 57,  1, 51, 63,  1, 44],
         [51, 43,  1, 42, 53,  1, 40, 39, 41, 49,  1, 56, 43, 41, 43, 47, 60, 43,
           1, 58, 46, 43,  1, 44, 50, 53, 59, 56,  1, 53, 44,  1],
         [58, 56, 59, 43, 12,  0,  0, 17, 24, 14, 27, 35, 10,  0, 27,  1, 58, 46,
          53, 59,  1, 41, 39, 47, 58, 47, 44, 44,  2,  1, 27,  1],
         [58, 46, 39, 58,  1, 58, 56, 59, 57, 58,  1, 58, 53,  1, 58, 46, 43, 43,
           8,  0,  0, 15, 13, 25, 21, 24, 24, 27, 10,  0, 20, 39],
         [49,  1, 58, 53,  1, 55, 59, 43, 52, 41, 46,  1, 47, 58,  6,  0, 31, 46,
 

In [19]:
model = GPT(vocab_size)
m = model.to(device)

In [20]:
print(sum(p.numel() for p in m.parameters()), 'M parameters')

209729 M parameters


In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [22]:
optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [23]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()


step 0: train loss 4.1828, val loss 4.1848
step 100: train loss 2.6217, val loss 2.6210
step 200: train loss 2.4995, val loss 2.4913
step 300: train loss 2.3977, val loss 2.4048
step 400: train loss 2.3174, val loss 2.3345
step 500: train loss 2.2660, val loss 2.2992
step 600: train loss 2.2099, val loss 2.2250
step 700: train loss 2.1672, val loss 2.1906
step 800: train loss 2.1346, val loss 2.1759
step 900: train loss 2.1026, val loss 2.1441
step 1000: train loss 2.0759, val loss 2.1054
step 1100: train loss 2.0511, val loss 2.1040
step 1200: train loss 2.0218, val loss 2.0998
step 1300: train loss 2.0116, val loss 2.0742
step 1400: train loss 1.9740, val loss 2.0551
step 1500: train loss 1.9556, val loss 2.0409
step 1600: train loss 1.9267, val loss 2.0410
step 1700: train loss 1.9246, val loss 2.0178
step 1800: train loss 1.9064, val loss 2.0087
step 1900: train loss 1.8975, val loss 2.0172
step 2000: train loss 1.8678, val loss 1.9993
step 2100: train loss 1.8909, val loss 2.0117


In [24]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


BUCKINGHAM:
Herefore misslaw tell sens'd in gike what kin men
By HERM:
ANCLE HORTIO:
I perly see hour
The so wonter in rybyt nobs no great his jontempter;
But stain letereth he here here;
'Tis being! down I that my plage, would with stand
The cannot shall minest sime Volsent,
Or hour in swraw me comphant earth.


CLart I:
He gursel?

Croriland:
Warrront, the here unsubt's.
That sweet to might kiss, jursty woldrove talk.

CORIS:
Bein their hearth; caid, what deviced insterpre's counselxship,
or will but as and he' incaustiest
That ears!
That she father of your cunal--


ESTBOLT:
Well, lay e'er wife; you duke.

Nurse, my lought deed I
can will myself quire, making Sick thereous;
Varen juy she-childow it, if be with homes you
That desostled
To him the hath not untongbred
This liful of sencount, prize!

LIS:
Har; yearswordsy, ray! what Like,
That your raid his reness on the cristerm
I counding with ble for head not?
Take bele be lover
That side thems sway kift hat the gading Of our by.

D